In [ ]:
import pandas as pd

# Load your dataset
df = pd.read_csv('data/raw/combined_iasi_no2_meteo_2020_2025_local.csv')

# Check for exact duplicates across all columns
exact_duplicates = df.duplicated().sum()
print(f"Exact duplicates found: {exact_duplicates}")

# View duplicate rows
duplicate_rows = df[df.duplicated(keep=False)]
print(f"Total rows involved in duplication: {len(duplicate_rows)}")

# Check duplicates on specific key columns
key_duplicates = df.duplicated(subset=['datetime']).sum()
print(f"Duplicates based on time: {key_duplicates}")

Exact duplicates found: 0
Total rows involved in duplication: 0
Duplicates based on sensor and time: 0


In [3]:
# Remove exact duplicates (keep first occurrence)
df_cleaned = df.drop_duplicates(keep='first')

# For key-based duplicates, investigate first
duplicate_keys = df[df.duplicated(subset=['datetime'], keep=False)]
print("Duplicate key patterns:")
print(duplicate_keys.groupby(['datetime']).size().sort_values(ascending=False))

# Remove duplicates based on key columns after investigation
df_cleaned = df.drop_duplicates(subset=['datetime'], keep='first')

Duplicate key patterns:
Series([], dtype: int64)


In [5]:
with open('outputs/logs/data_cleaning_log.txt', 'w') as f:
    f.write(f"Duplicate removal on {pd.Timestamp.now()}\n")
    f.write(f"Original dataset: {len(df)} rows\n")
    f.write(f"Exact duplicates removed: {exact_duplicates}\n")
    f.write(f"Key-based duplicates removed: {key_duplicates}\n")
    f.write(f"Final dataset: {len(df_cleaned)} rows\n")